In [1]:
import numpy as np
import pandas as pd
from textacy import text_stats, make_spacy_doc
from datasets import Dataset, load_dataset
import matplotlib.pyplot as plt
import itertools
import torch
import os

In [2]:
df_to_prep = pd.read_csv("Accepted_Rejected_with_scores.csv")

In [6]:
data_human = pd.DataFrame(load_dataset("argilla/ultrafeedback-binarized-preferences-cleaned")["train"]).sample(n=1000, random_state=42)

data_metric = df_to_prep.copy()

data_random = df_to_prep.copy()

## Human Data for fine tuning:

In [10]:
data_human['chosen'] = data_human['chosen'].apply(lambda x: x[1]['content'])
data_human['rejected'] = data_human['rejected'].apply(lambda x: x[1]['content'])
data_human = data_human.loc[:, ['prompt', 'chosen', 'rejected']]

## Metric Data for fine tuning:

In [14]:
data_metric

,Unnamed: 0,source,prompt,response,model,chosen,rating,n_sents,n_words,n_uniquewords,...,flesch_grade,gunning_fog,coleman_liau,automated_readability,x_cord,y_cord,topic_id,embeddings,metadata,topic_name
0,0,sharegpt,You are master of a tabletop role-playing game...,Okay.,gpt-3.5-turbo,0,2.50,1.0,1.0,1.0,...,-3.400000,0.400000,-21.868680,-2.090000,2.174568,11.309359,9,"[0.5708048939704895, -0.5634718537330627, 0.25...","('metadata', {'source': 'sharegpt', 'rating': ...",Multifaceted Experience Exploration
1,1,flan_v2_niv2,Teacher:You are given a review about a place. ...,"Based on the review, I would give Windsor a 5-...",wizardlm-13b,1,5.00,5.0,102.0,77.0,...,9.256196,12.081569,11.456347,11.765882,0.085421,9.718014,5,"[0.4610349237918854, 0.2453455775976181, -0.12...","('metadata', {'source': 'flan_v2_niv2', 'ratin...",Experiencing Food Options
2,2,sharegpt,How can I prepare for the onsite software engi...,"Research the company and position, prepare you...",alpaca-7b,0,3.50,1.0,40.0,31.0,...,20.660000,23.000000,16.680672,25.181500,3.320544,10.814219,1,"[-0.004047790076583624, 0.2616795301437378, -0...","('metadata', {'source': 'sharegpt', 'rating': ...",Business Market Analysis
3,3,flan_v2_niv2,"TASK DEFINITION: In this task, you are given a...",3)NaturalPlace\n\nConfidence: 80%,wizardlm-13b,1,3.25,1.0,3.0,3.0,...,17.046667,27.866667,25.295478,20.890000,0.944889,11.082933,6,"[0.21649840474128723, -0.29131922125816345, -0...","('metadata', {'source': 'flan_v2_niv2', 'ratin...",Comprehensive Movie Analysis
4,4,sharegpt,give me a more thorough understanding of each ...,To provide a thorough understanding of each ma...,gpt-3.5-turbo,0,2.75,2.0,56.0,37.0,...,11.133571,14.771429,8.971853,13.260357,2.448773,11.555860,9,"[0.713230550289154, 0.011790168471634388, -0.3...","('metadata', {'source': 'sharegpt', 'rating': ...",Multifaceted Experience Exploration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,sharegpt,given a list of transaction objects that conta...,Here is an example implementation of such a fu...,starchat,0,4.25,3.0,130.0,65.0,...,20.825385,24.410256,20.378555,29.764744,3.309534,9.679629,8,"[0.3024021089076996, 0.41639816761016846, -0.2...","('metadata', {'source': 'sharegpt', 'rating': ...",Programming Language Fundamentals
1996,1996,flan_v2_p3,More advanced farming practices allowed a sing...,Answer: This machine allows Farmer Bill to gro...,gpt-4,0,4.25,1.0,23.0,22.0,...,11.336522,14.417391,12.823339,14.029565,0.919685,12.876962,10,"[0.9053902626037598, 0.22944892942905426, -0.0...","('metadata', {'source': 'flan_v2_p3', 'rating'...",Complex Concepts
1997,1997,sharegpt,You are master of a tabletop role-playing game...,Okay. I understand the rules of the game. Let'...,mpt-30b-chat,1,4.75,8.0,72.0,62.0,...,6.275556,8.600000,9.984317,6.358333,2.174568,11.309359,9,"[0.5708048939704895, -0.5634718537330627, 0.25...","('metadata', {'source': 'sharegpt', 'rating': ...",Multifaceted Experience Exploration
1998,1998,flan_v2_p3,Please answer the following question: Please a...,Based on the information provided in the parag...,mpt-30b-chat,0,2.50,4.0,85.0,54.0,...,9.772794,12.735294,12.621157,13.077471,2.120657,14.088191,12,"[0.07037213444709778, 0.7733094096183777, 0.34...","('metadata', {'source': 'flan_v2_p3', 'rating'...",Social Impact Analysis
